In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key='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',
    model_name="llama-3.3-70b-versatile"
)
response=llm.invoke("The first person to land on moon was..")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.belzabar.com/jobs/senior-infrastructure-engineer-linux")
page_data= loader.load().pop().page_content
print(page_data)

Careers at | Belzabar Software Design India Pvt LtdBelzabar SoftwareHomeServicesAboutAbout UsTeamLife at BelzabarCareersConnectSenior DevOps EngineerSenior Infrastructure EngineerAs a Senior Infrastructure Engineer at BSDI, you will play a key role in bringing important software to market and into widespread use.In this role you will integrate a variety of leading-edge technology stacks in a secure manner, and ensure availability and safety of the production systems at scale. Software security analysis and considerations are a key part of this role.You have a thorough computer science background and experience with full stack software engineering and architecture, and have previously played a key role engineering enterprise-quality projects. You create solutions that are elegant, thoughtful and rigorous.You are hands-on and a team player, and combine camaraderie and mentorship to effectively deliver good functionality with efficient time-to-market. You communicate effectively and accur

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON formate containing
    following keys: "role","experience","skills" and "description".You have to give the information only once
    only return the valid JSON
    ### VALID JSON(NO PREAMBLE):
    """
)
chain_extract=prompt_extract | llm
res=chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Senior Infrastructure Engineer",
  "experience": "At least 6 years of experience in software infrastructure and engineering",
  "skills": [
    "Expert in Linux/UNIX",
    "Development experience in scripting language such as Perl, Python, PHP or Ruby",
    "Shell scripting",
    "Networking with experience in IPv4 and IPv6",
    "System and network monitoring and alerting systems",
    "IT security"
  ],
  "description": "As a Senior Infrastructure Engineer at BSDI, you will play a key role in bringing important software to market and into widespread use. You will integrate a variety of leading-edge technology stacks in a secure manner, and ensure availability and safety of the production systems at scale."
}
```


In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Infrastructure Engineer',
 'experience': 'At least 6 years of experience in software infrastructure and engineering',
 'skills': ['Expert in Linux/UNIX',
  'Development experience in scripting language such as Perl, Python, PHP or Ruby',
  'Shell scripting',
  'Networking with experience in IPv4 and IPv6',
  'System and network monitoring and alerting systems',
  'IT security'],
 'description': 'As a Senior Infrastructure Engineer at BSDI, you will play a key role in bringing important software to market and into widespread use. You will integrate a variety of leading-edge technology stacks in a secure manner, and ensure availability and safety of the production systems at scale.'}

In [15]:
type(json_res)

dict

In [18]:
import pandas as pd

df = pd.read_csv("links.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import chromadb
import uuid 

## client to store in memory but persistent client to store it in disk
client = chromadb.PersistentClient('Vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents= row["Techstack"],
                       metadatas={"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [23]:
links = collection.query(query_texts=["Experience in python","Expertise in React"],n_results=1).get("metadatas")
links

[[{'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'}]]

In [25]:
job = json_res
job["skills"]

['Expert in Linux/UNIX',
 'Development experience in scripting language such as Perl, Python, PHP or Ruby',
 'Shell scripting',
 'Networking with experience in IPv4 and IPv6',
 'System and network monitoring and alerting systems',
 'IT security']

In [27]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Teja Kukatla, a business development executive at Blynk. Blynk is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Blynk
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Blynk's portfolio: {link_list}
        Remember you are Teja Kukatla, BDE at blynk. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Infrastructure Engineering Solutions for BSDI

Dear Hiring Manager,

I came across the Senior Infrastructure Engineer role at BSDI and was impressed by the company's commitment to bringing innovative software to market. As a Business Development Executive at Blynk, I'd like to introduce our company's capabilities in providing tailored infrastructure engineering solutions that can support your goals.

At Blynk, we specialize in facilitating seamless integration of business processes through automated tools, leveraging our expertise in AI and software consulting. Our team of experts has a proven track record of empowering enterprises with scalable, optimized, and cost-effective solutions. We're confident that our capabilities align with your requirements for a Senior Infrastructure Engineer.

Our team excels in Linux/UNIX, scripting languages such as Python, and shell scripting, which are essential for integrating leading-edge technology stacks securely. We also have exte